## Test of Cell Tracking Application

In [1]:
from deepcell.applications import NuclearSegmentation, CytoplasmSegmentation, CellTracking
from deepcell.datasets import DynamicNuclearNetSample

import numpy as np
import tifffile
import imageio
import copy
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import os

key = 'POtyuCIN.nWwQ1FXgroiE8zKTWLgf5rdqGRJeKQHf'
os.environ.update({"DEEPCELL_ACCESS_TOKEN": key})

## Load Data
1. Dynamic Nuclear Fluorescence
2. Dynamic Collagen and Cytoplasm multichannel

In [11]:
x, y, _ = DynamicNuclearNetSample().load_data()
image_name = '12.20.17_combo2-cell1.tif'
cell_data = tifffile.imread(f'cell_track_data/{image_name}')

print(f"caliban data shape: {x.shape}, prediction shape: {y.shape}")
print(f"collagen data shape: {cell_data.shape}")

caliban data shape: (10, 540, 540, 1), prediction shape: (10, 540, 540, 1)
collagen data shape: (57, 3, 2, 210, 208)


### Reshape collagen data

DeepCell does not yet work with 3D segmentation, so we must pick one of the three z-slices.

We also notice that the shape of DeepCell data is (T, X, Y, C) whereas our data is in (T, Z, C, X, Y) so we must reshape

In [3]:
# Shuffle dimensions
cell_data = np.transpose(cell_data, (0,3,4,2,1))
print(f'after tranposing: {cell_data.shape}')

# Take one z-slice
z_pos = 2
cell_data = cell_data[..., z_pos]
print(f'single z-slice: {cell_data.shape}')

# Separate collagen and cytoplasm channel
collagen_data = np.expand_dims(cell_data[..., 0], 3)
cytoplasm_data = np.expand_dims(cell_data[..., 1], 3)
print(f'collagen shape: {collagen_data.shape}\ncytoplasm shape: {cytoplasm_data.shape}')

after tranposing: (57, 210, 208, 2, 3)
single z-slice: (57, 210, 208, 2)
collagen shape: (57, 210, 208, 1)
cytoplasm shape: (57, 210, 208, 1)


### Save images as gifs

In [40]:
def save_gif(x, filename, title):
    # ims = [Image.fromarray(x[i,...,0], 'LA') for i in range(x.shape[0])]
    # ims[0].save(filename, save_all=True, 
    #             append_images=ims[1:],
    #             optimize=False, loop=0 )
    
    def plot(im):
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(im, 'Greys_r', vmax=3000)
        plt.axis('off')
        plt.title(title)

        fig.canvas.draw()  # draw the canvas, cache the renderer
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

        plt.close(fig)

        return image

    imageio.mimsave(filename, [plot(x[i, ..., 0]) for i in range(x.shape[0])], loop=0)
    


save_gif(x, 'caliban-raw.gif', 'Caliban')
save_gif(cytoplasm_data, f'12.20.17_combo2-cell1-cyto-zpos{z_pos}.gif', 'Cytoplasm')

### Display saved 

<img src="raw_gifs/caliban-raw.gif" width="300pt" align="center" loop="infinite">
<img src="raw_gifs/12.20.17_combo2-cell1-cyto-zpos2.gif" width="300pt" align="center" loop="infinite">

## Methods for displaying models

In [9]:
def shuffle_colors(ymax, cmap):
    """Utility function to generate a colormap for a labeled image"""
    cmap = mpl.colormaps[cmap].resampled(ymax)
    nmap = cmap(range(ymax))
    np.random.shuffle(nmap)
    cmap = ListedColormap(nmap)
    cmap.set_bad('black')
    return cmap

In [10]:
def save_with_seg(x, y, filename):
    ymax = np.max(y)
    cmap = shuffle_colors(ymax, 'tab20')

    def plot(x, y):
        yy = copy.deepcopy(y)
        yy = np.ma.masked_equal(yy, 0)

        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        ax[0].imshow(x, cmap='Greys_r', vmax=3000)
        ax[0].axis('off')
        ax[0].set_title('Raw')
        ax[1].imshow(yy, cmap=cmap, vmax=ymax)
        ax[1].set_title('Segmented')
        ax[1].axis('off')

        fig.canvas.draw()  # draw the canvas, cache the renderer
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)

        return image

    imageio.mimsave(
        filename,
        [plot(x[i,...,0], y[i,...,0])
        for i in range(y.shape[0])],
        loop=0
    )


In [12]:
def save_with_tracked(x, y, filename):
    ymax = np.max(y)
    cmap = shuffle_colors(ymax, 'tab20')

    def plot(x, y):
        yy = copy.deepcopy(y)
        yy = np.ma.masked_equal(yy, 0)

        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        ax[0].imshow(x, cmap='Greys_r', vmax=3000)
        ax[0].axis('off')
        ax[0].set_title('Raw')
        ax[1].imshow(yy, cmap=cmap, vmax=ymax)
        ax[1].set_title('Tracked')
        ax[1].axis('off')

        fig.canvas.draw()  # draw the canvas, cache the renderer
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)

        return image

    imageio.mimsave(
        filename,
        [plot(x[i,...,0], y[i,...,0])
        for i in range(y.shape[0])],
        loop=0
    )

## Run Segmentation model on data to create masks

Before any cell tracking can happen, DeepCell requires masks to be created by its segmentation models.

In this section, we will also create stardist and cellpose masks to see if there is any overlap in compatibility.

In [ ]:
# DeepCell on caliban nuclear
deep_nuclear = NuclearSegmentation()
y_pred = deep_nuclear.predict(x, image_mpp=0.65)
# save_with_seg(x, y_pred, 'caliban-segmented.gif')

INFO:root:Checking for cached data
INFO:root:Checking NuclearSegmentation-75.tar.gz against provided file_hash...
INFO:root:NuclearSegmentation-75.tar.gz with hash efc4881db5bac23219b62486a4d877b3 already available.
INFO:root:Extracting C:\Users\aarus\.deepcell\models\NuclearSegmentation-75.tar.gz
INFO:root:Successfully extracted C:\Users\aarus\.deepcell\models\NuclearSegmentation-75.tar.gz into C:\Users\aarus\.deepcell\models


INFO:root:Converting image dtype to float


Produced:

<img src="deepcell_tracked/caliban-segmented.gif" width="600pt" align="center" loop="infinite">

In [6]:
# DeepCell on example cytoplasm data
deep_cyto = CytoplasmSegmentation()
deep_cyto_pred = deep_cyto.predict(cytoplasm_data, image_mpp=0.5)
# save_with_seg(cytoplasm_data, deep_cyto_pred, '12.20.17_combo2-cell1-cyto-zpos2-segmented.gif')

Produced:

<img src="deepcell_tracked/12.20.17_combo2-cell1-cyto-zpos2-segmented.gif" width="600pt" align="center" loop="infinite">

In [7]:
# Stardist on example cytoplasm data
from stardist.models import StarDist2D
from csbdeep.utils import normalize

sd_model = StarDist2D.from_pretrained('2D_versatile_fluo')
labels = [sd_model.predict_instances(normalize(item))[0] for item in cytoplasm_data]
sd_predict = np.expand_dims(np.array(labels), 3)
# save_with_seg(cytoplasm_data, sd_predict, 'stardist_seg.gif')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


Produced:

<img src="stardist_tracked/stardist_seg.gif" width="600pt" align="center" loop="infinite">

In [13]:
# Cellpose on example cytoplasm data
from cellpose import models

cyto3 = models.Cellpose(model_type='cyto3')
labels = [cyto3.eval(x, channels=[0,0])[0] for x in cytoplasm_data]
cellpose_predict = np.expand_dims(np.array(labels), 3)
# save_with_seg(cytoplasm_data, cellpose_predict, 'cellpose_seg.gif')

Produced:

<img src="cellpose_tracked/cellpose_seg.gif" width="600pt" align="center" loop="infinite">

## An exploration of DeepCell's cell tracking using different segmentions

DeepCell provides one model for cell tracking and the goal of this section is to see how accurate it is as well as if it can handle segmentations produced by other segmentation models/packages

In [9]:
tracker = CellTracking()

### DeepCell tracking

In [10]:
y_tracked = tracker.track(x, y_pred)
save_with_seg(x, y_tracked['y_tracked'], 'deepcell_tracked_nuclei.gif')

y_cytoplasm_tracked = tracker.track(cytoplasm_data, deep_cyto_pred)
save_with_seg(cytoplasm_data, y_cytoplasm_tracked['y_tracked'], 'deepcell_tracked/deepcell_tracked_cytoplasm.gif')

Results from DeepCell

<img src="deepcell_tracked/deepcell_tracked_nuclei.gif" width="600pt" align="center" loop="infinite">

<img src="deepcell_tracked/deepcell_tracked_cytoplasm.gif" width="600pt" align="center" loop="infinite">

### Stardist cytoplasm tracking

In [12]:
sd_tracked = tracker.track(cytoplasm_data, sd_predict)
save_with_seg(cytoplasm_data, sd_tracked['y_tracked'], f'stardist_tracked/{image_name}_tracked.gif')

Result from StarDist tracking

<img src="stardist_tracked/12.20.17_combo2-cell1.tif_tracked.gif" width="600pt" align="center" loop="infinite">

### Cellpose cytoplasm tracking

In [14]:
cellpose_tracked = tracker.track(cytoplasm_data, cellpose_predict)
save_with_seg(cytoplasm_data, cellpose_tracked['y_tracked'], f'cellpose_tracked/{image_name}_tracked.gif')

Cellpose tracking results

<img src="cellpose_tracked/12.20.17_combo2-cell1.tif_tracked.gif" width ="600pt" align="center">

In [11]:
image_names = ["12.20.17_combo2-cell1.tif", "12.20.17_combo2-cell2.tif", "12.20.17_combo2-cell6-1_fig1.tif", "12.20.17_combo2-cell7-1.tif"]

def track_with_deepcell(image_names):
    deep_cyto = CytoplasmSegmentation()
    tracker = CellTracking()
    for name in image_names:
        cell_data = tifffile.imread(f'cell_track_data/{name}')
        cell_data = np.transpose(cell_data, (0,3,4,2,1))
        # print(f'after tranposing: {cell_data.shape}')

        # Take one z-slice
        for z_pos in range(cell_data.shape[-1]):
            print(f'#### Processing {name}: {cell_data.shape} @ z = {z_pos} ####\n')
            cell_data_z = cell_data[..., z_pos]
            # print(f'single z-slice: {cell_data_z.shape}')

            # Separate collagen and cytoplasm channel
            cytoplasm_data = np.expand_dims(cell_data_z[..., 1], 3)
            # print(f'cytoplasm shape: {cytoplasm_data.shape}')
            deep_cyto_pred = deep_cyto.predict(cytoplasm_data, image_mpp=0.5)
            y_cytoplasm_tracked = tracker.track(cytoplasm_data, deep_cyto_pred)
            save_with_seg(cytoplasm_data, y_cytoplasm_tracked['y_tracked'], f'deepcell_tracked/Z{z_pos}-{name}.gif')

            

track_with_deepcell(image_names)


INFO:root:Checking for cached data
INFO:root:Checking NuclearTrackingNE-75.tar.gz against provided file_hash...
INFO:root:NuclearTrackingNE-75.tar.gz with hash a466682c9d1d5e3672325bb8a13ab3e0 already available.
INFO:root:Extracting C:\Users\aarus\.deepcell\models\NuclearTrackingNE-75.tar.gz
INFO:root:Successfully extracted C:\Users\aarus\.deepcell\models\NuclearTrackingNE-75.tar.gz into C:\Users\aarus\.deepcell\models


INFO:root:Checking for cached data
INFO:root:Checking NuclearTrackingInf-75.tar.gz against provided file_hash...
INFO:root:NuclearTrackingInf-75.tar.gz with hash 5dbd8137be851a0c12557fcde5021444 already available.
INFO:root:Extracting C:\Users\aarus\.deepcell\models\NuclearTrackingInf-75.tar.gz
INFO:root:Successfully extracted C:\Users\aarus\.deepcell\models\NuclearTrackingInf-75.tar.gz into C:\Users\aarus\.deepcell\models


INFO:root:Converting image dtype to float


#### Processing 12.20.17_combo2-cell1.tif: (57, 210, 208, 2, 3) @ z = 0 ####



INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 10.
INFO:CellTracker:Created new track for cell 11.
INFO:CellTracker:Tracked frame 1 in 0.48648500000012973 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 12.
INFO:CellTracker:Tracked frame 2 in 0.4778757999999925 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Created new track for cell 13.
INFO:CellTracker:Tracked frame 3 in 0.4790599000000384 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Tracked frame 4 in 0.05068679999999404 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Created new track for cell 14.
INFO:CellTracker:Created new track for cell 15.
INFO:CellTracker:Tracked frame 5 in 0.05005839999989803 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Created new track for cell 16.
INFO:CellTracker:Tracked frame 6 in 0.05027190000009796 s.
INFO:CellTracker:Tracking frame 7
INFO:CellTracker:Created new track for cell 17.
INFO:CellTracker:Created n

...

INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 15.
INFO:CellTracker:Created new track for cell 16.
INFO:CellTracker:Created new track for cell 17.
INFO:CellTracker:Created new track for cell 18.
INFO:CellTracker:Created new track for cell 19.
INFO:CellTracker:Created new track for cell 20.
INFO:CellTracker:Tracked frame 1 in 0.08242479999989882 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 21.
INFO:CellTracker:Created new track for cell 22.
INFO:CellTracker:Tracked frame 2 in 0.06839159999981348 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Created new track for cell 23.
INFO:CellTracker:Created new track for cell 24.
INFO:CellTracker:Created new track for cell 25.
INFO:CellTracker:Tracked frame 3 in 0.0653201000000081 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 26.
INFO:CellTracker:Created new track for cell 27.
INFO:CellTracker:Tracked frame 4 in 0.06288549999999304 s.
INFO:

#### Processing 12.20.17_combo2-cell7-1.tif: (57, 238, 232, 2, 2) @ z = 0 ####



INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 9.
INFO:CellTracker:Tracked frame 1 in 0.060629500000004555 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 10.
INFO:CellTracker:Created new track for cell 11.
INFO:CellTracker:Tracked frame 2 in 0.06521510000015951 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Created new track for cell 12.
INFO:CellTracker:Created new track for cell 13.
INFO:CellTracker:Tracked frame 3 in 0.060700799999949595 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Tracked frame 4 in 0.05405760000007831 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Created new track for cell 14.
INFO:CellTracker:Tracked frame 5 in 0.05500340000003234 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Created new track for cell 15.
INFO:CellTracker:Tracked frame 6 in 0.05910549999998693 s.
INFO:CellTracker:Tracking frame 7
INFO:CellTracker:Created new track for cell 16.
INFO:CellTracker:Create

#### Processing 12.20.17_combo2-cell7-1.tif: (57, 238, 232, 2, 2) @ z = 1 ####



INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 11.
INFO:CellTracker:Created new track for cell 12.
INFO:CellTracker:Tracked frame 1 in 0.06457819999991443 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 13.
INFO:CellTracker:Created new track for cell 14.
INFO:CellTracker:Tracked frame 2 in 0.060177600000088205 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Created new track for cell 15.
INFO:CellTracker:Created new track for cell 16.
INFO:CellTracker:Tracked frame 3 in 0.061065300000109346 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 17.
INFO:CellTracker:Tracked frame 4 in 0.05824829999983194 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Created new track for cell 18.
INFO:CellTracker:Tracked frame 5 in 0.05859050000003663 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Tracked frame 6 in 0.05447249999997439 s.
INFO:CellTracker:Tracking frame 7
INFO:CellTracker:Track

In [13]:
from stardist.models import StarDist2D
from csbdeep.utils import normalize

sd_model = StarDist2D.from_pretrained('2D_versatile_fluo')
tracker = CellTracking()

Found model '2D_versatile_fluo' for 'StarDist2D'.


INFO:root:Checking for cached data
INFO:root:Checking NuclearTrackingNE-75.tar.gz against provided file_hash...
INFO:root:NuclearTrackingNE-75.tar.gz with hash a466682c9d1d5e3672325bb8a13ab3e0 already available.
INFO:root:Extracting C:\Users\aarus\.deepcell\models\NuclearTrackingNE-75.tar.gz
INFO:root:Successfully extracted C:\Users\aarus\.deepcell\models\NuclearTrackingNE-75.tar.gz into C:\Users\aarus\.deepcell\models


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


INFO:root:Checking for cached data
INFO:root:Checking NuclearTrackingInf-75.tar.gz against provided file_hash...
INFO:root:NuclearTrackingInf-75.tar.gz with hash 5dbd8137be851a0c12557fcde5021444 already available.
INFO:root:Extracting C:\Users\aarus\.deepcell\models\NuclearTrackingInf-75.tar.gz
INFO:root:Successfully extracted C:\Users\aarus\.deepcell\models\NuclearTrackingInf-75.tar.gz into C:\Users\aarus\.deepcell\models


In [15]:
def track_with_stardist(image_names):
    for name in image_names:
        cell_data = tifffile.imread(f'cell_track_data/{name}')
        cell_data = np.transpose(cell_data, (0,3,4,2,1))
        # print(f'after tranposing: {cell_data.shape}')

        # Take one z-slice
        for z_pos in range(cell_data.shape[-1]):
            print(f'#### Processing {name}: {cell_data.shape} @ z = {z_pos} ####\n')
            cell_data_z = cell_data[..., z_pos]
            # print(f'single z-slice: {cell_data_z.shape}')

            # Separate collagen and cytoplasm channel
            cytoplasm_data = np.expand_dims(cell_data_z[..., 1], 3)
            # print(f'cytoplasm shape: {cytoplasm_data.shape}')

            labels = [sd_model.predict_instances(normalize(item))[0] for item in cytoplasm_data]
            sd_predict = np.expand_dims(np.array(labels), 3)
            sd_tracked = tracker.track(cytoplasm_data, sd_predict)
            save_with_tracked(cytoplasm_data, sd_tracked['y_tracked'], f'stardist_tracked/{name}-Z{z_pos}.gif')

            

track_with_stardist(image_names)

#### Processing 12.20.17_combo2-cell1.tif: (57, 210, 208, 2, 3) @ z = 0 ####



INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 4.
INFO:CellTracker:Tracked frame 1 in 0.679078600000139 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 5.
INFO:CellTracker:Tracked frame 2 in 0.6205335999998169 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Tracked frame 3 in 0.6403273000000809 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 6.
INFO:CellTracker:Created new track for cell 7.
INFO:CellTracker:Tracked frame 4 in 0.06478099999958431 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Tracked frame 5 in 0.056222300000172254 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Tracked frame 6 in 0.05184330000020054 s.
INFO:CellTracker:Tracking frame 7
INFO:CellTracker:Created new track for cell 8.
INFO:CellTracker:Created new track for cell 9.
INFO:CellTracker:Detected division! Cell 9 is daughter of cell 5.
INFO:CellTracker:Tracked frame 7 in 0.060565999999653286 s.
INF

#### Processing 12.20.17_combo2-cell1.tif: (57, 210, 208, 2, 3) @ z = 1 ####



INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 7.
INFO:CellTracker:Tracked frame 1 in 0.05134269999962271 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Tracked frame 2 in 0.05406360000006316 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Tracked frame 3 in 0.052653100000043196 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 8.
INFO:CellTracker:Tracked frame 4 in 0.05333269999982804 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Tracked frame 5 in 0.053126000000247586 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Tracked frame 6 in 0.052531700000145065 s.
INFO:CellTracker:Tracking frame 7
INFO:CellTracker:Created new track for cell 9.
INFO:CellTracker:Detected division! Cell 9 is daughter of cell 4.
INFO:CellTracker:Tracked frame 7 in 0.05624320000015359 s.
INFO:CellTracker:Tracking frame 8
INFO:CellTracker:Created new track for cell 11.
INFO:CellTracker:Tracked frame 8 in 0.05347430000028908 s

#### Processing 12.20.17_combo2-cell1.tif: (57, 210, 208, 2, 3) @ z = 2 ####



...

INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 21.
INFO:CellTracker:Created new track for cell 22.
INFO:CellTracker:Created new track for cell 23.
INFO:CellTracker:Tracked frame 1 in 0.07315799999969386 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 24.
INFO:CellTracker:Created new track for cell 25.
INFO:CellTracker:Tracked frame 2 in 0.0671173999999155 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Created new track for cell 26.
INFO:CellTracker:Created new track for cell 27.
INFO:CellTracker:Tracked frame 3 in 0.0696444000000156 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 28.
INFO:CellTracker:Tracked frame 4 in 0.06875450000006822 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Created new track for cell 29.
INFO:CellTracker:Tracked frame 5 in 0.06705670000019381 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Created new track for cell 30.
INFO:CellTracker:Create

#### Processing 12.20.17_combo2-cell7-1.tif: (57, 238, 232, 2, 2) @ z = 0 ####



INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 12.
INFO:CellTracker:Tracked frame 1 in 0.06437039999991612 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Tracked frame 2 in 0.05799180000030901 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Tracked frame 3 in 0.061993100000108825 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 13.
INFO:CellTracker:Tracked frame 4 in 0.05810219999966648 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Tracked frame 5 in 0.055941099999927246 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Created new track for cell 14.
INFO:CellTracker:Tracked frame 6 in 0.05815339999981006 s.
INFO:CellTracker:Tracking frame 7
INFO:CellTracker:Created new track for cell 15.
INFO:CellTracker:Tracked frame 7 in 0.09192030000031082 s.
INFO:CellTracker:Tracking frame 8
INFO:CellTracker:Created new track for cell 16.
INFO:CellTracker:Tracked frame 8 in 0.06594760000007227 s.
INFO:CellTrack

#### Processing 12.20.17_combo2-cell7-1.tif: (57, 238, 232, 2, 2) @ z = 1 ####



INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 13.
INFO:CellTracker:Detected division! Cell 13 is daughter of cell 3.
INFO:CellTracker:Created new track for cell 14.
INFO:CellTracker:Tracked frame 1 in 0.06365000000005239 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 16.
INFO:CellTracker:Tracked frame 2 in 0.061793500000021595 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Created new track for cell 17.
INFO:CellTracker:Created new track for cell 18.
INFO:CellTracker:Created new track for cell 19.
INFO:CellTracker:Created new track for cell 20.
INFO:CellTracker:Created new track for cell 21.
INFO:CellTracker:Detected division! Cell 21 is daughter of cell 4.
INFO:CellTracker:Tracked frame 3 in 0.07604259999970964 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 23.
INFO:CellTracker:Tracked frame 4 in 0.062392799999997806 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Created n

In [ ]:
def track_with_cellpose(image_names):
    for name in image_names:
        cell_data = tifffile.imread(f'cell_track_data/{name}')
        cell_data = np.transpose(cell_data, (0,3,4,2,1))
        # print(f'after tranposing: {cell_data.shape}')

        # Take one z-slice
        for z_pos in range(cell_data.shape[-1]):
            print(f'#### Processing {name}: {cell_data.shape} @ z = {z_pos} ####\n')
            cell_data_z = cell_data[..., z_pos]
            # print(f'single z-slice: {cell_data_z.shape}')

            # Separate collagen and cytoplasm channel
            cytoplasm_data = np.expand_dims(cell_data_z[..., 1], 3)
            # print(f'cytoplasm shape: {cytoplasm_data.shape}')

            labels = [sd_model.predict_instances(normalize(item))[0] for item in cytoplasm_data]
            sd_predict = np.expand_dims(np.array(labels), 3)
            sd_tracked = tracker.track(cytoplasm_data, sd_predict)
            save_with_tracked(cytoplasm_data, sd_tracked['y_tracked'], f'stardist_tracked/{name}-Z{z_pos}.gif')

            

track_with_stardist(image_names)